# Scoring functions: many parameters and outside data
    
### Note: To run this notebook, you need an API Key. You can get one [here](mailto:charles.brecque@mindfoundry.ai)    
More tutorials are [available here](./)

In [60]:
from mindfoundry.optaas.client.client import OPTaaSClient, Goal
from mindfoundry.optaas.client.parameter import FloatParameter, Distribution
client = OPTaaSClient('<URL>', "<API key>")

## Adding multiple parameters at once
- Instead of adding parameters one by one, we can of course create a list.
- When writing the scoring function, we can use a dict() to iterate over the parameters.

In [62]:
param_names = ['x1', 'x2', 'x3', 'x4']

params = []
for name in param_names:
    params.append(FloatParameter(name, minimum=-5, maximum=5, default=0, id=name, distribution=Distribution.UNIFORM))

In [63]:
# A simple problem, maximize the sum of 4 upside-down quadratics. All shifted by 2. So the optimum is at [2, 2, 2, 2]
def scoring_function(**params):
    score = 10
    for param_name, param_value in params.items():  # iterate over each parameter, its name and value
        score = score - (param_value - 2) ** 2
    return score

In [64]:
from mindfoundry.optaas.client.client import Goal

task = client.create_task(
    title='Advanced Parameter Task',
    parameters=params,
    goal=Goal.max
)

In [65]:
best_result = task.run(scoring_function, max_iterations=15, score_threshold=32)
print("Best Result:", best_result)

Running task "Advanced Parameter Task" for 15 iterations
(or until score is 32 or better)

Iteration: 0    Score: -6
Configuration: {'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0}

Iteration: 1    Score: -31.0
Configuration: {'x1': 2.5, 'x2': -2.5, 'x3': 2.5, 'x4': -2.5}

Iteration: 2    Score: -31.0
Configuration: {'x1': -2.5, 'x2': 2.5, 'x3': -2.5, 'x4': 2.5}

Iteration: 3    Score: -44.75
Configuration: {'x1': -1.25, 'x2': -1.25, 'x3': 1.25, 'x4': -3.75}

Iteration: 4    Score: -29.75
Configuration: {'x1': 3.75, 'x2': 3.75, 'x3': -3.75, 'x4': 1.25}

Iteration: 5    Score: -44.75
Configuration: {'x1': 1.25, 'x2': -3.75, 'x3': -1.25, 'x4': -1.25}

Iteration: 6    Score: -29.75
Configuration: {'x1': -3.75, 'x2': 1.25, 'x3': 3.75, 'x4': 3.75}

Iteration: 7    Score: -46.3125
Configuration: {'x1': -3.125, 'x2': -1.875, 'x3': -1.875, 'x4': 1.875}

Iteration: 8    Score: -18.8125
Configuration: {'x1': 1.875, 'x2': 3.125, 'x3': 3.125, 'x4': -3.125}

Iteration: 9    Score: -43.8125
Configuration: {'x1'

## Using outside data in a scoring function
- Let's say we want to have a scoring function same as above but with varying shifts for the quadratic
- We can access outside information by wrapping the scoring function
- 'x_shifts' is the outside information, it cannot be modified during optimization

In [47]:
def make_scoring_function(x_shifts): 
    def scoring_function(**params): 
        score = 10
        for param_name, param_value in params.items():
            score = score - (param_value - x_shifts[param_name]) ** 2
        return score
    
    return scoring_function

In [48]:
# Each quadratic will be shifted by a custom amount. So the new optimum should be at [0.2, 0.5, 0.3, 0.0]
param_x_shifts = {'x1': 0.2, 'x2': 0.5, 'x3': 0.3, 'x4': 0.0}

scoring_function_with_x_shifts = make_scoring_function(param_x_shifts)  # param_x_shifts are now part of our scoring function

In [51]:
from mindfoundry.optaas.client.client import Goal

task = client.create_task(
    title='Quick Start Example Task',
    parameters=params, # Parameters, same as above
    goal=Goal.max
)

In [52]:
best_result = task.run(scoring_function_with_x_shifts, max_iterations=15, score_threshold=32)
print("Best Result:", best_result)

Running task "Quick Start Example Task" for 15 iterations
(or until score is 32 or better)

Iteration: 0    Score: -0.38
Configuration: {'x1': 0, 'x2': 0, 'x3': 0, 'x4': 0}

Iteration: 1    Score: -25.38
Configuration: {'x1': 2.5, 'x2': -2.5, 'x3': 2.5, 'x4': -2.5}

Iteration: 2    Score: -25.38
Configuration: {'x1': -2.5, 'x2': 2.5, 'x3': -2.5, 'x4': 2.5}

Iteration: 3    Score: -20.13
Configuration: {'x1': -1.25, 'x2': -1.25, 'x3': 1.25, 'x4': -3.75}

Iteration: 4    Score: -41.129999999999995
Configuration: {'x1': 3.75, 'x2': 3.75, 'x3': -3.75, 'x4': 1.25}

Iteration: 5    Score: -23.13
Configuration: {'x1': 1.25, 'x2': -3.75, 'x3': -1.25, 'x4': -1.25}

Iteration: 6    Score: -42.13
Configuration: {'x1': -3.75, 'x2': 1.25, 'x3': 3.75, 'x4': 3.75}

Iteration: 7    Score: -24.9425
Configuration: {'x1': -3.125, 'x2': -1.875, 'x3': -1.875, 'x4': 1.875}

Iteration: 8    Score: -27.4425
Configuration: {'x1': 1.875, 'x2': 3.125, 'x3': 3.125, 'x4': -3.125}

Iteration: 9    Score: -60.4425
C

In [53]:
param_x_shifts

{'x1': 0.2, 'x2': 0.5, 'x3': 0.3, 'x4': 0.0}

In [ ]:
# The optimization got fairly close to the optimum